## Imports

In [1]:
# imports
import pandas as pd
import altair as alt


## Read and Manipulate Data

In [2]:
# read in aggregated bike traffic data
df_traffic = (pd.read_excel("data/traffic/cleaned/aggregated-combined-traffic-data.xlsx")).fillna(0)

# show dataframe
df_traffic

,Intersection,Time,YR_2016,YR_2017,YR_2018,Total
0,6,01:00 AM,4.571429,0.00,0.0,4.571429
1,6,01:00 PM,57.142857,0.00,0.0,57.142857
2,6,02:00 AM,2.928571,0.00,0.0,2.928571
3,6,02:00 PM,60.642857,0.00,0.0,60.642857
4,6,03:00 AM,1.714286,0.00,0.0,1.714286
...,...,...,...,...,...,...
91,1071,10:00 PM,5.333333,3.00,4.5,4.428571
92,1071,11:00 AM,5.000000,5.75,8.0,6.071429
93,1071,11:00 PM,2.833333,1.75,1.5,2.142857
94,1071,12:00 AM,2.666667,2.75,1.0,2.214286


In [3]:
# rename cols
df_traffic = df_traffic.rename(columns={'YR_2016':'2016', 'YR_2017': "2017", 'YR_2018':'2018'})

# transform data into long form
df_traffic = df_traffic.melt(id_vars=["Intersection", "Time"], value_vars = ["2016","2017","2018"], var_name="Year", 
                    value_name="Traffic")

# show dataframe
df_traffic

,Intersection,Time,Year,Traffic
0,6,01:00 AM,2016,4.571429
1,6,01:00 PM,2016,57.142857
2,6,02:00 AM,2016,2.928571
3,6,02:00 PM,2016,60.642857
4,6,03:00 AM,2016,1.714286
...,...,...,...,...
283,1071,10:00 PM,2018,4.500000
284,1071,11:00 AM,2018,8.000000
285,1071,11:00 PM,2018,1.500000
286,1071,12:00 AM,2018,1.000000


In [4]:
# add latitude and longitude column to df_traffic based on intersection
df_traffic['Latitude'] = df_traffic['Intersection'].map({6: '42.351986', 451: '42.340097', 1071: '42.32693', 601: '42.33693'})
df_traffic['Longitude'] = df_traffic['Intersection'].map({6: '-71.090079', 451: '-71.081539', 1071: '-71.06684', 601: '-71.07806'})

# show dataframe
df_traffic

,Intersection,Time,Year,Traffic,Latitude,Longitude
0,6,01:00 AM,2016,4.571429,42.351986,-71.090079
1,6,01:00 PM,2016,57.142857,42.351986,-71.090079
2,6,02:00 AM,2016,2.928571,42.351986,-71.090079
3,6,02:00 PM,2016,60.642857,42.351986,-71.090079
4,6,03:00 AM,2016,1.714286,42.351986,-71.090079
...,...,...,...,...,...,...
283,1071,10:00 PM,2018,4.500000,42.32693,-71.06684
284,1071,11:00 AM,2018,8.000000,42.32693,-71.06684
285,1071,11:00 PM,2018,1.500000,42.32693,-71.06684
286,1071,12:00 AM,2018,1.000000,42.32693,-71.06684


## Background Map Image

In [5]:
# define source df for altair to use
source_bike_network = pd.DataFrame.from_records([
    {"x":  0, "y": 0,
     "img": "data/bike_network_data/bike_lanes_2.jpeg"}
])

# altair chart for bike network background
bike_network = alt.Chart(source_bike_network).mark_image(
    width=500,
    height=500
).encode(
    x='x',
    y='y', 
    url='img'
)

# show chart
bike_network

/Users/sree/anaconda3/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

## Scatterplot of Bike Counts

In [6]:
# make scatter plot of bike counts at each intersection
traffic = alt.Chart(df_traffic).transform_aggregate(
    sum_traffic='sum(Traffic)',
    groupby=['Latitude', 'Longitude']
).mark_circle(
    fill='red', # make all circles red
).encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    #color=alt.Color('Year:N'), # color by year for linking
    size=alt.Size('sum_traffic:Q', scale=alt.Scale(range=[0, 200]), legend=alt.Legend(title='Bike Count', 

        # legend configuration based on final chart
        orient='none',
        legendX=425, legendY=-80,
        direction='horizontal',
        titleAnchor='middle')),
    tooltip=['sum_traffic:Q']
)

# show chart
traffic

alt.Chart(...)

## Layer Plot on Map

In [7]:
# combine bike network map image and bike traffic scatter plot
traffic_map = bike_network + traffic

# add title
traffic_map = traffic_map.properties(
    title='Bike Traffic on 9/27/16 at 12 AM'
)

# show plot
traffic_map

alt.LayerChart(...)

# Bike Lane Legend

In [8]:
# define source df of bike lanes legend for altair to use
source_bike_legend = pd.DataFrame.from_records([
    {"x":  0, "y": 0,
     "img": "data/bike_network_data/bike_lane_legend.png"}
])

# altair chart for bike lanes legend
bike_network_legend = alt.Chart(source_bike_legend).mark_image(
    width=400,
    height=400
).encode(
    x=alt.X('x', axis=None),
    y=alt.Y('y', axis=None), 
    url='img'
)

# show chart
bike_network_legend

/Users/sree/anaconda3/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

## Concatenate for Final Static Chart

In [9]:
# hconcatenate legend with bike traffic map chart
final = alt.hconcat(bike_network_legend, traffic_map)

# show plot and remove axis behind legend
final.configure_view(strokeWidth=0) 

alt.HConcatChart(...)